In [1]:
# Let's start learning
import torch
from torch_geometric.datasets import Planetoid
from cora_loader import CitationNetwork,CocitationNetwork
from torch.utils.data import DataLoader
from torch_geometric.nn import GCNConv,SAGEConv,GATConv,GraphConv,SGConv

import torch.nn.functional as F

from torch_geometric_node2vec import Node2Vec

from multi_layered_model import MonoModel,BiModel,TriModel
import time
# from models_eval import eval_multiple,eval_multiple_bi,eval_multiple_tri,eval_multiple_tripre

In [2]:
citeseer = CitationNetwork('/tmp/citeseer','citeseer',directed=False)
cora = CitationNetwork('/tmp/cora','cora',directed=False)

In [3]:
coraDirected = CitationNetwork('/tmp/coraDirected','cora',directed=True)
coraReversed = CitationNetwork('/tmp/coraReversed','cora',directed=True,reverse=True)
citeseerDirected = CitationNetwork('/tmp/citeseerDirected','citeseer',directed=True)
citeseerReversed = CitationNetwork('/tmp/citeseerReversed','citeseer',directed=True,reverse=True)

In [6]:
def run_and_eval_model(dataset,channels1,arch,modelType,epochs=200):
    # training process (without batches/transforms)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = arch(modelType,dataset,channels1).to(device)
    data = dataset[0].to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4)
    model.train() # to enter training phase
    for epoch in range(epochs):
        optimizer.zero_grad() # saw this a lot in the beginning, maybe resetting gradients (not to accumulate)
        out = model(data) # this calls the forward method apparently
        loss = F.nll_loss(out[data.train_mask],data.y[data.train_mask]) # nice indexing, easy and short
        loss.backward() # magic: real back propagation step, takes care of the gradients and stuff
        optimizer.step() # maybe updates the params to be optimized
    model.eval() # enter eval phase
    _,pred = model(data).max(dim=1) # take prediction out of softmax
    correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / data.test_mask.sum().item()
    return acc

def eval_multiple(dataset,channels1,arch,modelType,runs=100,epochs=50):
    start = time.time()
    accs = []
    for i in range(runs):
        accs.append(run_and_eval_model(dataset,channels1,arch,modelType,epochs))
    elapsed_time = time.time() - start
    print('Elaplsed {}'.format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
    return sum(accs)/len(accs),accs


In [9]:
eval_multiple(coraDirected,[16],MonoModel,GCNConv,runs=10,epochs=50)[0]

Elaplsed 00:00:02


0.20639999999999997

In [10]:
eval_multiple(cora,[16],MonoModel,GCNConv,runs=10,epochs=50)[0]

Elaplsed 00:00:04


0.35540000000000005

In [12]:
eval_multiple(cora,[16,16,16],MonoModel,GCNConv,runs=10,epochs=50)[0]

Elaplsed 00:00:03


0.27280000000000004